# Making an example NEO4J db for the talk

In [12]:
import random

## Creating the nodes data

In [13]:
resource_codes = []
scale = 1
for i in range(0,50*scale):
    resource_codes.append("Res_" + str(i))

In [14]:
group_codes = []
for i in range(0,250*scale):
    group_codes.append("G_" + str(i))

In [15]:
person_codes = []
for i in range(0,1500*scale):
    person_codes.append("P_" + str(i))

# Creating the edges randomly

In [16]:
person_is_in_group = []


# Assign every person to a group
for person in person_codes:
    person_is_in_group.append([person,random.choice(group_codes)])
    
# Assign a 50% of people to a random set of groups:

for person in random.sample(person_codes, int(round(len(person_codes)/2))):
    for extra_group in range(0,random.randint(1, 5)):
        person_is_in_group.append([person,random.choice(group_codes)])
    

In [17]:
group_child_of_group = []

for i in range(0, len(group_codes)):
    group_child_of_group.append([random.choice(group_codes),random.choice(group_codes)])

In [18]:
group_has_access_resource = []

for resource in resource_codes:
    group_has_access_resource.append([random.choice(group_codes),resource])


## Insert the data into Neo4j

In [19]:
from neo4j import GraphDatabase

uri = "bolt://localhost:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "q1w2"))

### Importing the nodes into NEO4J

In [20]:
with driver.session() as session:
    tx = session.begin_transaction()
    for resource in resource_codes:
        tx.run("CREATE (r:Resource {code:'"+resource+"'}) RETURN r.name")
    tx.commit()

In [21]:
with driver.session() as session:
    tx = session.begin_transaction()
    for group in group_codes:
        tx.run("CREATE (g:Group {code:'"+group+"'}) RETURN g.name")
    tx.commit()

In [22]:
with driver.session() as session:
    tx = session.begin_transaction()
    for person in person_codes:
        tx.run("CREATE (p:Person {code:'"+person+"'}) RETURN p.name")
    tx.commit()

### Importing the edges into NEO4J

In [23]:
with driver.session() as session:
    tx = session.begin_transaction()
    for edge in person_is_in_group:
        tx.run(
            """
            MATCH (p:Person {code:'"""+edge[0]+"""'})
            MATCH (g:Group {code:'"""+edge[1]+"""'})
            CREATE (p)-[:IS_IN]->(g) RETURN p,g
            """
            
        )
    tx.commit()

In [24]:
with driver.session() as session:
    tx = session.begin_transaction()
    for edge in group_child_of_group:
        tx.run(
            """
            MATCH (g1:Group {code:'"""+edge[0]+"""'})
            MATCH (g2:Group {code:'"""+edge[1]+"""'})
            CREATE (g1)-[:CHILD_OF]->(g2) RETURN g1,g2
            """
            
        )
    tx.commit()

In [25]:
with driver.session() as session:
    tx = session.begin_transaction()
    for edge in group_has_access_resource:
        tx.run(
            """
            MATCH (g:Group {code:'"""+edge[0]+"""'})
            MATCH (r:Resource {code:'"""+edge[1]+"""'})
            CREATE (g)-[:HAS_ACCESS]->(r) RETURN g,r
            """
            
        )
    tx.commit()